<!DOCTYPE html>
<html>

<body>
    <h2><span style="color: lightblue;">Influx v3 - Insert High Cardinality Data</span></h2>
    <h3><strong>Desc:</strong></h3>
    <ul>
        <li>Generate dummy data using Python Pandas.</li>
        <li>Convert the data to CSV.</li>
        <li>Insert the CSV to Influx.</li>
    </ul>
    <h3>Comments:</h3>
    <ul>
        <li>This notebook is still in <strong>DRAFT</strong> mode.</li>
        <li>This notebook was developed for Influx cloud v3. Do NOT use it with v2</li>
    </ul>
</body>
</html>

In [1]:
## Connect to Influx v3 
import os, time
from influxdb_client_3 import InfluxDBClient3, Point
import certifi # we need it to support their certification problems. 
import pandas as pd
from datetime import timedelta

token = "PjObG70ZggAu78U5hN9awHq5pOk6GOtAsu7Fp4JROzdHmllRCMrlQh0r9owylgKR0A_Ki7-EYgzz03TNad0tqw=="
org = "Metis-v3"
host = "https://eu-central-1-1.aws.cloud2.influxdata.com"
bucket = "test"

try: 
    client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())
except ExceptionType as e:
    print (str(e))

<!DOCTYPE html>
<html>
<body>
    <h3><span style="color: lightblue;">View the Exisintg data</span></h3>
    <ul>
        <li>The size of the Measurement (num of rows)</li>
        <li>Numebr of items in each series.</li>
    </ul>
</body>
</html>

In [2]:
query = """
SELECT COUNT(*) FROM QUERY_DETAILS
"""
table = client.query(query=query, database=bucket, mode ="all", language="influxql")
dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
print (dataframe)

## Some usefule commands ( https://docs.influxdata.com/influxdb/v1/query_language/explore-schema/)
## SHOW RETENTION POLICIES;
## SHOW MEASUREMENTS;
## SELECT calls, host FROM QUERY_DETAILS LIMIT 10;  I think you must select at least one measure or it returns an empty results set
## Notice - The SELECT should NOT contain the columns again (weird, I know). It shows the columns, includig time, automatically, based on your selection. 
query = """
SELECT mean(calls) 
FROM QUERY_DETAILS 
WHERE database_name = 'airbases'
AND time >=  '2023-10-01 00:00:00'
AND host = 'database-2.cofhrj7zmyn4.eu-central-1.rds.amazonaws.com'
GROUP BY  query_id, time(60m);
"""

query = """
SELECT * 
FROM DS_INDEXES_ACTIVITY
LIMIT 100;
"""
table = client.query(query=query, database=bucket, mode ="all", language="influxql")
dataframe = table.to_pandas() # This one automatically eliminitaes the NULL values. Not good. 
print (dataframe)



  iox::measurement       time  count_calls  count_rows  count_total_exec_time  \
0    QUERY_DETAILS 1970-01-01     32768959    32768959               32768959   

   count_value  
0     32768959  
       iox::measurement                    time      apiKey             db  \
0   DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest       airbases   
1   DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest       airbases   
2   DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest       airbases   
3   DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest       airbases   
4   DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest       airbases   
..                  ...                     ...         ...            ...   
95  DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest  airbases-demo   
96  DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest  airbases-demo   
97  DS_INDEXES_ACTIVITY 2023-10-24 12:34:43.751  newvertest  airbases-demo   
98  DS_INDEXES_ACTIVITY

## Generate a Data Frame with the dummy data. 
Step 1: A single Query ID, last 3 days, group hourly. 

In [81]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Number of hosts, databases, and tables
num_hosts = 1
num_databases_per_host = 1
num_tables_per_database = 200
num_days = 3

# Arrays of hosts and databases
hosts = [f'host_{i}' for i in range(1, num_hosts + 1)]
databases = [f'db_{i}' for i in range(1, num_databases_per_host * num_hosts + 1)]

data = {'time': [], 'host_id': [], 'db_id': [], 'table_name': [], 'rows_read': [], 'rows_inserted': [],
        'rows_deleted': [], 'rows_updated': []}

# Iterate over hosts
for host_id in hosts:
    # Iterate over databases
    for db_id in databases:
        start_date = datetime(2023, 10, 1)

        # Generate data for the DataFrame
        for table_id in range(1, num_tables_per_database + 1):
            for day in range(num_days):
                for minute in range(0, 24 * 60, 5):
                    timestamp = start_date + timedelta(days=day, minutes=minute)

                    # Append data to the DataFrame
                    data['time'].append(timestamp)
                    data['host_id'].append(host_id)
                    data['db_id'].append(db_id)
                    data['table_name'].append(f'table_{table_id}')
                    data['rows_read'].append(np.random.randint(0, 1000))
                    data['rows_inserted'].append(np.random.randint(0, 500))
                    data['rows_deleted'].append(np.random.randint(0, 200))
                    data['rows_updated'].append(np.random.randint(0, 300))

# Create the DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV file
df.to_csv('database_activity_data_last_3_days.csv', index=False)


In [82]:
print("Number of rows in the DataFrame:", len(df))
print("DataFrame size (rows, columns):", df.shape)
# Save DataFrame to CSV file

 172,800,000

Number of rows in the DataFrame: 172800
DataFrame size (rows, columns): (172800, 8)
